In [2]:
import pandas as pd 
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import os
from datetime import datetime, timedelta

In [3]:
PATH = 'C:\\Education\\teamproject\\final_prioject\\news\\종목코드.csv' #전역변수 , 상수처리는 (MUST HAVE TO THAT) CAPITAL로  FINAL(값을 못바꾼다)

In [10]:
data1 = pd.read_csv('./종목코드.csv')
data1['종목명'],data1['종목코드']

(0              3S
 1             APS
 2             CBI
 3      CJ 바이오사이언스
 4           CJ씨푸드
           ...    
 937            흥국
 938        흥국에프엔비
 939          흥국화재
 940            희림
 941            힘스
 Name: 종목명, Length: 942, dtype: object,
 0       60310
 1       54620
 2       13720
 3      311690
 4       11150
         ...  
 937     10240
 938    189980
 939       540
 940     37440
 941    238490
 Name: 종목코드, Length: 942, dtype: object)

In [11]:
data1['종목명'].shape

(942,)

In [19]:
#절대경로를 쓰자 : 파일 위치가 바껴서 csv 파일은 한 폴더에 몰아놓고 절대 안옮길 파일들을 

def crawler(company_code, start_date, end_date):
    page = 1
    
    while True:
        url = 'https://finance.naver.com/item/news_news.nhn?code=' + str(company_code) + '&page=' + str(page)
        source_code = requests.get(url).text
        html = BeautifulSoup(source_code, "lxml")
        
        titles = html.select('.title')
        title_result = [title.get_text(strip=True) for title in titles]
        
        links = html.select('.title')
        link_result = ['https://finance.naver.com' + link.find('a')['href'] for link in links]
        
        dates = html.select('.date')
        date_result = [date.get_text(strip=True) for date in dates]
                                                                            #문자열   
        # 현재 날짜로부터 1년 6개월 전의 날짜 계산
        #one_and_a_half_year_ago = datetime.now() - timedelta(days=1.8*365/2)
        
        # 예외 처리를 통해 날짜 형식이 일치하지 않으면 다시 입력받도록 수정
        # try:
        #     date_result = [datetime.strptime(date.replace('.', ''), '%Y%m%d') for date in date_result]
        # except ValueError:
        #     print("잘못된 날짜 형식입니다. 'YYYY.MM.DD' 또는 'YYYYMMDD' 형식으로 다시 입력해주세요.")
        #     return
        
        # 시작 날짜부터 1년 6개월 전의 뉴스만 가져오도록 필터링
        result = {"날짜": date_result, "기사제목": title_result, "링크": link_result}
        df_result = pd.DataFrame(result)
        df_result = df_result[(df_result['날짜'] >= start_date) & (df_result['날짜'] <= end_date)]
                                                                                    #DT 로 만들기  start_data 를 문자열로 바꿔야 형식을 맞춰서 비교 해야함 
                                                                                    
        if not df_result.empty:
            print("다운 받고 있습니다------")
            df_result.to_csv(f'page{page}.csv', mode='w', encoding='utf-8-sig', index=False)
            page += 1
        else:
            print("더 이상 뉴스가 없습니다.")
            break

def convert_to_code(company, start_date, end_date):
    # 가정한 데이터프레임 생성
    data1 = pd.read_csv(PATH)
    data1
    # data = pd.DataFrame({'종목명': ['삼성전자', 'LG화학'], '종목코드': ['005930', '051910']})
    
    # 데이터프레임의 열 이름을 확인하고 수정하세요.
    company_name_column = data1['종목명']
    company_code_column = data1['종목코드']
    
    keys = company_name_column.tolist()
    values = company_code_column.tolist()
    
    dict_result = dict(zip(keys, values))
    
    pattern = '[a-zA-Z가-힣]+' 
    
    if bool(re.match(pattern, company)) == True:
        company_code = dict_result.get(str(company))
        crawler(company_code, start_date, end_date)
    else:
        company_code = str(company)
        crawler(company_code, start_date, end_date)

def main():
    info_main = input("="*50+"\n"+"실시간 뉴스기사 다운받기."+"\n"+" 시작하시려면 Enter를 눌러주세요."+"\n"+"="*50)
    
    company = input("종목명이나 코드 입력: ") 
    end_date = input("종료 날짜(YYYY.MM.DD 또는 YYYYMMDD) 입력: ")
    
    # 예외 처리를 통해 날짜 형식이 일치하지 않으면 다시 입력받도록 수정
    try:
        end_date = datetime.strptime(end_date.replace('.', ''), '%Y%m%d').date()
    except ValueError:
        print("잘못된 날짜 형식입니다. 'YYYY.MM.DD' 또는 'YYYYMMDD' 형식으로 다시 입력해주세요.")
        return
    # 시작 날짜를 현재 날짜로부터 1년 6개월 전으로 설정
    start_date = (datetime.now() - timedelta(days=1.8*365/2)).date()
    
    print(start_date, end_date)
    convert_to_code(company, start_date, end_date)


In [20]:
main()

2023-03-07 2023-11-30


TypeError: '>=' not supported between instances of 'str' and 'datetime.datetime'